In [11]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata

# Einlesen der CSV-Datei
df = pd.read_csv('../op_players_list.csv')

# Filtern der Spieler mit über 450 Minuten Spielzeit
df = df[df['Minutes played'] > 450]

# Funktionen zur Berechnung pro 90 Minuten Werte
def per_90(stat, minutes):
    return (stat / minutes) * 90 if minutes > 0 else 0

# Berechnung der pro 90 Minuten Werte für alle relevanten Statistiken
for col in df.columns:
    if col not in ['Player', 'Nation', 'Age', 'Club', 'Position', 'Minutes played', '90 minutes played']:
        df[f'{col} per 90'] = df.apply(lambda row: per_90(row[col], row['Minutes played']), axis=1)

# Berechnung der Perzentile für jede Position
positions = df['Position'].unique()
percentiles = {}

for pos in positions:
    pos_df = df[df['Position'] == pos]
    percentiles[pos] = pos_df.copy()
    
    for col in df.columns:
        if col.endswith('per 90'):
            percentiles[pos][f'{col} percentile'] = rankdata(pos_df[col], method='average') / len(pos_df) * 100

# Zusammenführen der Perzentile in den ursprünglichen DataFrame
for pos, pos_df in percentiles.items():
    for col in pos_df.columns:
        if col.endswith('per 90 percentile'):
            df.loc[df['Position'] == pos, col] = pos_df[col]

# Filtern der Daten für Virgil van Dijk
van_dijk = df[df['Player'] == 'Rajiv van La Parra'].iloc[0]

# Identifizierung der Kategorien mit den höchsten Perzentilen für Virgil van Dijk
van_dijk_stats = van_dijk.filter(like='percentile')
top_stats = van_dijk_stats.sort_values(ascending=False).head(20)

print("Top Kategorien für Virgil van Dijk:")
print(top_stats)


/var/folders/zx/_6c6bk211jg76gggnhvmrt900000gn/T/ipykernel_63042/151034475.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col} per 90'] = df.apply(lambda row: per_90(row[col], row['Minutes played']), axis=1)
/var/folders/zx/_6c6bk211jg76gggnhvmrt900000gn/T/ipykernel_63042/151034475.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col} per 90'] = df.apply(lambda row: per_90(row[col], row['Minutes played']), axis=1)
/var/folders/zx/_6c6bk211jg76gggnhvmrt900000gn/T/ipykernel_63042/151034475.py:18: PerformanceW

IndexError: single positional indexer is out-of-bounds